# Assignment for 2025/11/04
You will need to generate a plot and upload it on coospace.

Bagi Laura: Humán idegsejtek dendrittípus szerinti ('spiny' 'aspiny' 'sparsely spiny') nyugalmi membránpotenticál (vrest) eloszlásdiagramja.

Bélai Emese Krisztina: Humán idegsejtek disease_state szerinti nyugalmi membránpotenticál (vrest) eloszlása (hisztogramja).

Gyuricza Gergő Pál: sag érték eloszlása egér jobb és bal agyféltekei neuronokban (structure_hemisphere)

Ignácz Rebeka: sag vs. fast_trough_v_short_square pontábrája, denrittipus szerint szinezve (dendrite_type)

Illés Patrícia Zsófia: sag vs. vrest denrittipus szerinti pontábrája.

Klein Kata:fast_trough_t_short_square eloszlása Vip-IRES-Cre és Pvalb-IRES-Cre transzgenikus egérvonalak szerint (transgenic_line).

Környei Viktor: Humán idegsejtek disease_state szerinti fast_trough_v_short_square eloszlása.

Kristóffy Zsolt János: bemenő ellenállás (input_resistance_mohm) vs reobázikus áram (rheobase_sweep_number) pontábrája, 3 és 5 rétegi (structure_layer_name) humán sejtekben

Lüvi Adél: sag (sag) vs nyugalmi membránpotenciál (vrest) pontábrája, 2/3 és 5 rétegi (structure_layer_name) egér sejtekben

Machó Fanni: agykérgi mélység (normalized_depth) vs. bemenő ellenállás (input_resistance_mohm) humán sejtekben

Micsinai Erika Manyi: agykérgi mélység (normalized_depth) vs. bemenő ellenállás (input_resistance_mohm) egér sejtekben

Nagy Vivien: tüzelési adaptáció (adaptation) eloszlása humán és egér 5. Rétegi (structure_layer_name) sejtekben

Pataki Zsombor Botond: agykérgi mélység (normalized_depth) vs. bemenő ellenállás (input_resistance_mohm) human sejtekben

Pethő Tibor: Norbert:bemenő ellenállás (input_resistance_mohm) vs fast_trough_v_short_square

Sarusi Ákos:nyugalmi membránpotenticál (vrest) eloszlása Vip-IRES-Cre és Pvalb-IRES-Cre transzgenikus egérvonalak szerint (transgenic_line).

Sásdi Bálint Bendegúz:sag eloszlása Vip-IRES-Cre és Sst-IRES-Cre transzgenikus egérvonalak szerint (transgenic_line).

Szafián Dorottya Anna: sag vs. adaptation denrittipus szerinti pontábrája.

Szatmári Rebeka:Humán idegsejtek disease_state szerinti nyugalmi membránpotenticál (vrest) eloszlása.

Temple Ramóna: avg_isi érték eloszlása egér jobb és bal agyféltekei neuronokban (structure_hemisphere)

Zagyva Renáta: sag vs. adaptation pontábrája human mintakban.


In [1]:
# Step 1: import the Allen SDK
try:
    import allensdk
    print('allensdk imported')
except ImportError as e:
    !pip install allensdk "numpy == 1.26.4" "pandas == 2.3.0" "matplotlib > 3.8.0" "statsmodels >= 0.14.4"

# you can check the version of the allensdk

allensdk imported


In [2]:
# Step 2: set up a directory where you save your data (feel free to use the google drive from the celltypes lesson)

import os
from pathlib import Path

save_path = '/content/Analyzing-Open-Neuroscience-Data-Course-SZTE/'   # set this to wherever you want to save your data

# check if folder already exists; if not, make it
# if it already exists, let us know
if os.path.isdir(save_path):
   print("Given directory exists")
else:
   print("Given directory doesn't exist, making it")
   Path(save_path).mkdir(parents=True,exist_ok=True)# make the folder so it exists

# change into the folder for the session and use it to save our figures
os.chdir(save_path)
print("Changed into directory")

print('data will be saved to {}'.format(save_path))

# check out your google drive!!

Given directory doesn't exist, making it
Changed into directory
data will be saved to /content/Analyzing-Open-Neuroscience-Data-Course-SZTE/


In [5]:
# Step 3: Import important packages, and redefine the get_sweep function
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import h5py # this is necessary for the get_sweep function below

# the get_sweep function reads the .npy files that you download

def get_sweep(self, sweep_number):
  """ Retrieve the stimulus, response, index_range, and sampling rate
  for a particular sweep.  This method hides the NWB file's distinction
  between a "Sweep" and an "Experiment".  An experiment is a subset of
  of a sweep that excludes the initial test pulse.  It also excludes
  any erroneous response data at the end of the sweep (usually for
  ramp sweeps, where recording was terminated mid-stimulus).

  Some sweeps do not have an experiment, so full data arrays are
  returned.  Sweeps that have an experiment return full data arrays
  (include the test pulse) with any erroneous data trimmed from the
  back of the sweep.

  Parameters
  ----------
  sweep_number: int

  Returns
  -------
  dict
      A dictionary with 'stimulus', 'response', 'index_range', and
      'sampling_rate' elements.  The index range is a 2-tuple where
      the first element indicates the end of the test pulse and the
      second index is the end of valid response data.
  """
  with h5py.File(self.file_name, 'r') as f:

      swp = f['epochs']['Sweep_%d' % sweep_number]

      # fetch data from file and convert to correct SI unit
      # this operation depends on file version. early versions of
      #   the file have incorrect conversion information embedded
      #   in the nwb file and data was stored in the appropriate
      #   SI unit. For those files, return uncorrected data.
      #   For newer files (1.1 and later), apply conversion value.
      major, minor = self.get_pipeline_version()
      if (major == 1 and minor > 0) or major > 1:
          # stimulus
          stimulus_dataset = swp['stimulus']['timeseries']['data']
          conversion = float(stimulus_dataset.attrs["conversion"])
          stimulus = stimulus_dataset[()] * conversion
          # acquisition
          response_dataset = swp['response']['timeseries']['data']
          conversion = float(response_dataset.attrs["conversion"])
          response = response_dataset[()] * conversion
      else:   # old file version
          stimulus_dataset = swp['stimulus']['timeseries']['data']
          stimulus = stimulus_dataset[()]
          response = swp['response']['timeseries']['data'][()]

      if 'unit' in stimulus_dataset.attrs:
          unit = stimulus_dataset.attrs["unit"].decode('UTF-8')

          unit_str = None
          if unit.startswith('A'):
              unit_str = "Amps"
          elif unit.startswith('V'):
              unit_str = "Volts"
          assert unit_str is not None, Exception(
              "Stimulus time series unit not recognized")
      else:
          unit = None
          unit_str = 'Unknown'

      swp_idx_start = swp['stimulus']['idx_start'][()]
      swp_length = swp['stimulus']['count'][()]

      swp_idx_stop = swp_idx_start + swp_length - 1
      sweep_index_range = (swp_idx_start, swp_idx_stop)

      # if the sweep has an experiment, extract the experiment's index
      # range
      try:
          exp = f['epochs']['Experiment_%d' % sweep_number]
          exp_idx_start = exp['stimulus']['idx_start'][()]
          exp_length = exp['stimulus']['count'][()]
          exp_idx_stop = exp_idx_start + exp_length - 1
          experiment_index_range = (exp_idx_start, exp_idx_stop)
      except KeyError:
          # this sweep has no experiment.  return the index range of the
          # entire sweep.
          experiment_index_range = sweep_index_range

      assert sweep_index_range[0] == 0, Exception(
          "index range of the full sweep does not start at 0.")

      return {
          'stimulus': stimulus,
          'response': response,
          'stimulus_unit' : unit_str,
          'index_range': experiment_index_range,
          'sampling_rate': 1.0 * swp['stimulus']['timeseries']['starting_time'].attrs['rate']
      }


In [69]:
#Step 3: download the data
from allensdk.core.cell_types_cache import CellTypesCache

#Initialize the cache as 'ctc' (cell types cache)
ctc = CellTypesCache(manifest_file='cell_types/manifest.json') #


In [70]:
# Step 4:

# Download the ephys features, convert to dataframe, get the metadata (cells variable), make a big dataframe (full_dataframe)
# hint: you can re-use the code from the class (as always)

# ephys_features = ctc.....
# ef_df = pd.DataFrame(....
# ...
# full_dataframe = ...



In [76]:
# Step 5:
# make a separate human and mouse dataframe, if it is your assignment (using the species column)

#mouse_df = full_dataframe[...
#human_df = full_dataframe[...

# Step 6:
# make further separations if necessary..


#Step 7: make a scatter plot, histogram, both, depending on your specific assignment
